In [ ]:
!pip install pandas numpy yfinance gradio scikit-learn

import pandas as pd
import numpy as np
import yfinance as yf
import gradio as gr
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import warnings

warnings.filterwarnings('ignore')

# Initializing MinMax Scalers
features_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler = MinMaxScaler(feature_range=(0, 1))

# Define function for Gradio Interface
def predict_stock_price(stock_ticker, close, ma_10, ma_50, model_choice):
    # Fetch stock data using yfinance
    stock_data = yf.download(stock_ticker, start="2023-01-01", end="2024-01-01")
    stock_data.reset_index(inplace=True)

    # Feature Engineering
    stock_data['MA_10'] = stock_data['Close'].rolling(window=10).mean()
    stock_data['MA_50'] = stock_data['Close'].rolling(window=50).mean()
    stock_data.dropna(inplace=True)

    # Selecting features and target
    X = stock_data[['Close', 'MA_10', 'MA_50']].values
    Y = stock_data['Close'].values.reshape(-1, 1)

    # Normalizing Features
    X = features_scaler.fit_transform(X)
    Y = target_scaler.fit_transform(Y)

    # Splitting dataset
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

    # Training Models
    rf_model = RandomForestRegressor(n_estimators=100, max_depth=10)
    rf_model.fit(X_train, y_train.ravel())

    gb_model = GradientBoostingRegressor(n_estimators=100, max_depth=10)
    gb_model.fit(X_train, y_train.ravel())

    input_data = np.array([[close, ma_10, ma_50]])
    input_data = features_scaler.transform(input_data)

    if model_choice == "Random Forest":
        predicted_price = rf_model.predict(input_data)
    else:
        predicted_price = gb_model.predict(input_data)

    predicted_price = target_scaler.inverse_transform(predicted_price.reshape(-1, 1)).ravel()[0]

    # Generate Prediction vs Actual Graph
    y_pred_test = rf_model.predict(X_test) if model_choice == "Random Forest" else gb_model.predict(X_test)
    y_pred_test = target_scaler.inverse_transform(y_pred_test.reshape(-1, 1))
    y_test_actual = target_scaler.inverse_transform(y_test.reshape(-1, 1))

    plt.figure(figsize=(10, 5))
    plt.plot(y_test_actual, label="Actual Price", color="blue", alpha=0.6)
    plt.plot(y_pred_test, label="Predicted Price", color="red", alpha=0.8)
    plt.legend()
    plt.xlabel("Days")
    plt.ylabel("Stock Price")
    plt.title(f"{model_choice} - Predicted vs Actual Prices")
    plt.grid(True)
    plt.savefig("prediction_graph.png")
    plt.close()

    # Feature Importance Graph for Random Forest
    if model_choice == "Random Forest":
        feature_importances = rf_model.feature_importances_
        feature_names = ['Close', 'MA_10', 'MA_50']

        plt.figure(figsize=(8, 5))
        plt.barh(feature_names, feature_importances, color="green", alpha=0.7)
        plt.xlabel("Feature Importance")
        plt.ylabel("Feature")
        plt.title("Random Forest Feature Importance")
        plt.savefig("feature_importance.png")
        plt.close()

    return f"Predicted Stock Price: ${predicted_price:.2f}", "prediction_graph.png", "feature_importance.png"

# Gradio Interface
iface = gr.Interface(
    fn=predict_stock_price,
    inputs=[
        gr.Textbox(label="Stock Ticker", placeholder="Enter stock ticker (e.g., AAPL)"),
        gr.Number(label="Closing Price"),
        gr.Number(label="10-Day Moving Average"),
        gr.Number(label="50-Day Moving Average"),
        gr.Radio(["Random Forest", "Gradient Boosting"], label="Select Model")
    ],
    outputs=["text", "image", "image"],
    title="ESG Risk Analysis And Stock Market Prediction",
    description="Enter stock details to predict future prices using machine learning models. "
                "The graphs display actual vs predicted prices and feature importance for Random Forest."
)

# Launch Gradio app
iface.launch()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.3 MB/s eta 0:00:00
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a4c1fd8b9cbe4477bb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/sp